In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import theano.tensor as tt
import pymc3 as pm
import pickle

import a as constants

In [3]:
# Customizations
sns.set(context="notebook", style="darkgrid")

# matplolibrc
plt.rcParams["figure.figsize"] = (12, 8)
#plt.rcParams["font.family"] = "serif"
#plt.rcParams["text.usetex"] = True
%config InlineBackend.figure_format = "retina"

# Load datas

In [4]:
t_x = np.loadtxt("Input/Xray.dat", usecols=0)
tr6g = np.loadtxt("Input/R6g.dat", usecols=0)
tr3g = np.loadtxt("Input/R3g.dat", usecols=0)

F_x = np.loadtxt("Input/Xray.dat", usecols=1)
F_r6g = np.loadtxt("Input/R6g.dat", usecols=1)
F_r3g = np.loadtxt("Input/R3g.dat", usecols=1)

In [5]:
Gamma = 490.
gama0b = 1.5
t = t_x * constants.day    # 160 * day #  1.0 * sec
tr3 = tr3g * constants.day # 160 * day #  1.0 * sec
tr6 = tr6g * constants.day # 160 * day #  1.0 * sec

z = 0.01 # Redshift, fixed parameter
D = 2.0 * np.power(10.0, 26.0) * constants.cm  # Luminosity distance, fixed parameter
niter = 7000 
tune = 3000
kk = 1

aa = 1.4 #0.5
bb = 1.5 #0.3
cc = 1.3 #0.25 
dd = 2.15 #063
ll = 1.0

In [6]:
with pm.Model() as Radio6GHz_model:
    
    # Prior Informations
    eta0 = pm.Normal(r'$n \ (10^{-4}cm^{-3})$', mu=1.0, sd=0.1)
    E_off0 = pm.Normal(r'$E_{off} \ (10^{49}erg)$', mu=3.0, sd=0.1)
    E_0 = pm.Normal(r'$\tilde{E} \ (10^{49}erg)$', mu=5.0, sd=0.1)
    
    dtheta1 = pm.Uniform(r'$\Delta\theta \ (deg)$', lower=15.5, upper=16.5)
    dtheta1a = pm.Uniform(r'$\theta_j \ (deg)$', lower=2.5, upper=3.5)
    #dtheta1 = pm.HalfNormal(r'$\Delta\theta \ (deg)$', sd=10.0)
    #dtheta1a = pm.HalfNormal(r'$\theta_j \ (deg)$', sd=10.0)
    
    
    alpha = pm.Normal(r'$p$', mu=2.25, sd=0.1)
    delta_1 = pm.Normal(r'$\alpha_S$', mu=2.3, sd=0.1)
    xiBf0 = pm.Normal(r'$\epsilon_B\ (10^{-4})$', mu=2.0, sd=0.1)
    xief0 = pm.Normal(r'$\epsilon_e \ (10^{-1}) $', mu=1.0, sd=0.1)
    sigma = pm.HalfNormal(r'$\sigma$', sd=10.0)
    
    # Relationships
    dtheta = pm.Deterministic('delta_theta', dtheta1*np.pi/180.0)
    theta_j = pm.Deterministic('theta_j', dtheta1a*np.pi/180.0) 
    delta = pm.Deterministic('delta', 4.0+delta_1)
    eta=pm.Deterministic('eta', eta0*1e-4*pow(constants.cm, -3.0))
    E0 = pm.Deterministic('E0', E_0*1e49*constants.erg)  
    E_off=pm.Deterministic('E_off', E_off0*1e49*constants.erg)
    xiBf=pm.Deterministic('xiBf', xiBf0*1e-3)
    xief=pm.Deterministic('xief', xief0*1e-1)
    xf = pm.Deterministic('xf', (-1.0+pow(1.0+4.0*xief/xiBf, 1.0/2.0))/2.0)
    
    # Functions
    def tdx_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta):
        A1 = pow(3.0 / (2.0 * np.pi * constants.mp), 1.0/3.0)
        return 0.5 * kk * A1 * pow(1.0+z, 1.0) * pow(eta, -1.0/3.0) * pow(E, 1.0/3.0) * pow(dtheta, 2.0) * 1.0 / constants.day

    def Gammax_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta, theta_j):
        A1 = pow(32.0*np.pi*constants.mp/3.0, -0.5) 
        return A1 * pow(1.0+z, 3.0/2.0) * pow(eta, -0.5) * pow(E, 0.5) * pow(theta_j, -1.0) * pow(dtheta, 3.0) * pow(t, -3.0/2.0)

    def gammam_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta, theta_j):
        A1 = pow(32.0*np.pi*constants.mp/3.0, -0.5) * constants.mp /constants.me * (alpha - 2.0) / (alpha - 1.0)
        return A1 * pow(1.0+z, 3.0/2.0) * xief * pow(eta, -0.5) * pow(E, 0.5) * pow(dtheta, 3.0) * pow(theta_j, -1.0) * pow(t, -3.0/2.0)

    def gammac_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta, theta_j):
        A1 = pow(3.0*np.pi/(32.0*constants.mp), 0.5) * constants.me / constants.sigmaT
        nn = 1.0
        xf = (-1.0 + pow(1.0+4.0*nn*xief/xiBf, 0.5)) * 0.5
        return A1 * pow(1.0+xf, -1.0) * pow(1.0+z, -0.5) * pow(xiBf, -1.0) * pow(eta, -0.5) * pow(E, -0.5) * pow(dtheta, -1.0) * pow(theta_j, 1.0) * pow(t, 0.5)

    def xff_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta, theta_j):
        gam_m_jet = gammam_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta,theta_j)
        gam_c_jet = gammac_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta,theta_j)
        nn = pow(gam_c_jet/gam_m_jet, 2.0-alpha)
        return (-1.0 + pow(1.0+4.0*nn*xief/xiBf, 0.5)) * 0.5

    def Em_bb_jet(xiBf, xief, gama0b, E, alpha, eta, D, z, t, delta, dtheta, theta_j): 
        # This module calculates the characteristic frecuency and give the value in Hz
        A1 = 3.0 * pow(2.0, 0.5) * constants.e * pow(constants.mp, 3.0/2.0) / (8.0 * pow(np.pi, 3.0/2.0) * pow(constants.me, 3.0)) * pow((alpha-2.0)/(alpha-1.0), 2.0)
        return A1 * pow(1.0+z, 2.0) * pow(xief, 2.0) * pow(xiBf, 0.5) * pow(eta, -0.5) * pow(E, 1.0) * pow(dtheta, 4.0) * pow(theta_j, -2.0) * pow(t, -3.0) * 1.0 / constants.GHz * 1.0 / 1.48

    def Ec_bb_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta, theta_j):
        # This module calculates the cut off frecuency and give the value in Hz
        A1 = 3.0 * np.pi * constants.e * pow(constants.me, 1.0) / (pow(constants.sigmaT, 2.0) * pow(32.0 * np.pi * constants.mp, 0.5))
        xf = xff_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta,theta_j)
        return A1 * pow(1.0+z,-2.0) * pow(1+xf, -2.0) * pow(xiBf, -3.0/2.0) * pow(eta,-0.5) * pow(E, -1.0) * pow(dtheta, -4.0) * pow(theta_j, 2.0) * pow(t, 1.0) * 1.0 / constants.KeV * 1.48

    def Fmax_bb_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta, theta_j):
        # This module calculates the cut off frecuency and give the value in Hz
        A1 = 64.0 * constants.sigmaT * constants.me * pow(32.0 * np.pi * constants.mp, 3.0/2.0) / (27.0 * constants.e)
        return A1 * pow(1.0+z, -4.0) * pow(xiBf, 0.5) * pow(eta, 2.5) * pow(D, -2.0) * pow(E, -1.0) * pow(dtheta, -18.0) * pow(theta_j, 2.0) * pow(t, 6.0) * 1.0 / constants.mJy * 1.0 / pow(1.48, 2.0) * 1.0 / (2.0 * np.pi)

    def Em_ab_jet(xiBf, xief, gama0b, E, alpha, eta, D, z, t, delta):
        # This module calculates the characteristic frecuency and give the value in Hz
        A1 = pow(8.0, 0.5) * constants.e * pow(constants.mp, 2.5) / (pow(np.pi, 0.5) * pow(constants.me, 3.0)) * pow((alpha-2.0)/(alpha-1.0), 2.0) * pow(3.0/(2.0*np.pi * constants.mp), 2.0/3.0)
        return A1 * pow(1.0+z, 1.0) * pow(xief, 2.0) * pow(xiBf, 0.5) * pow(eta, -1.0/6.0) * pow(E, 2.0/3.0) * pow(t, -2.0) * 1.0 / constants.GHz * 1.0 / 1.48

    def Ec_ab_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta):
        # This module calculates the cut off frecuency and give the value in Hz
        A1 = 18.0 * np.pi * constants.e * constants.me / (pow(constants.sigmaT, 2.0) * pow(32.0*np.pi*constants.mp, 3.0/2.0)) * pow(3.0/(2.0*np.pi*constants.mp), -2.0/3.0)
        xf = xff_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta, theta_j)
        return A1 * pow(1.0+z, -1.0) * pow(1+xf, -2.0) * pow(xiBf, -3.0/2.0) * pow(eta, -5.0/6.0) * pow(E, -2.0/3.0) * 1.0 / constants.KeV * 1.48 * 1.5

    def Fmax_ab_jet(xiBf, xief, E, alpha, eta, D, z, t, delta, dtheta):
        # This module calculates the cut off frecuency and give the value in Hz
        A1 = constants.sigmaT * constants.me * pow(32.0*np.pi*constants.mp, 0.5) / (9.0 * constants.e) * pow(3.0/(2.0*np.pi*constants.mp), 5.0/3.0)
        return pow(kk, 2.0) * A1 * pow(1.0+z, 4.0) * pow(xiBf, 0.5) * pow(eta, -1.0/6.0) * pow(D, -2.0) * pow(E, 5.0/3.0) * pow(t, -2.0) * 1.0 / constants.mJy * 1.0 / pow(1.48, 2.0) * 1.0 / (2.0 * np.pi) * 0.4

    def gammam_coc(xiBf, xief, E1, alpha, eta, D, z, t, delta, dtheta):
        A1 = pow(3.0/(32.0*np.pi*constants.mp), 1.0/(delta+8.0)) * constants.mp / constants.me * (alpha - 2.0) / (alpha - 1.0)
        return A1 * xief * pow(1.0+z, 3.0/(delta+8.0)) * pow(eta, -1.0/(delta+8.0) ) * pow(E1, 1.0/(delta+8.0)) * pow(t, -3.0/(delta+8.0))

    def gammac_coc(xiBf, xief, E1, alpha, eta, D, z, t, delta, dtheta):
        A1 = pow(3.0/(32.0*np.pi*constants.mp), -3.0/(delta+8.0)) * 3.0 * constants.me / (16.0 * constants.mp * constants.sigmaT)
        nn = 1.0
        xf = (-1.0 + pow(1.0+4.0*nn*xief/xiBf, 0.5)) * 0.5
        return A1 * pow(xiBf, -1.0) * pow(1.0+xf, -1.0) * pow(1.0+z, (delta-1.0)/(delta+8.0)) * pow(eta, -(delta+5.0)/(delta+8.0)) * pow(E1, -3.0/(delta+8.0)) * pow(t, (1.0-delta)/(delta+8.0))

    def xff_coc(xiBf, xief, E1, alpha, eta, D, z, t, delta, dtheta):
        gam_m_coc = gammam_coc(xiBf, xief, E1, alpha, eta, D, z, t, delta, dtheta)
        gam_c_coc = gammac_coc(xiBf, xief, E1, alpha, eta, D, z, t, delta, dtheta)
        nn = pow(gam_c_coc/gam_m_coc, 2.0-alpha)
        return (-1.0 + pow(1.0+4.0*nn*xief/xiBf, 0.5)) * 0.5

    def tdx_coc(xiBf, xief, E1, alpha, eta, D, z, t, delta, dtheta1):
        A1 = pow(3.0/(32.0*np.pi*constants.mp), 1.0/3.0)
        dtheta1 = 10.0 * 3.1416 / 180.0
        return 0.5 * kk * A1 * pow(1.0+z, 1.0) * pow(eta, -1.0/3.0) * pow(E1, 1.0/3.0) * pow(dtheta1, (delta+6.0)/3.0) * 1.0 / constants.day

    def Em_bb_coc(xiBf, xief, gama0b, E1, alpha, eta, D, z, t, delta, dtheta):
        # This module calculates the characteristic frecuency and give the value in Hz
        A1 = pow(2.0, 0.5) * constants.e * pow(constants.mp, 2.5) / (pow(np.pi, 0.5) * pow(constants.me, 3.0)) * pow((alpha-2.0)/(alpha-1.0), 2.0)
        A2 = pow(3.0/(32.0*np.pi*constants.mp), 4.0/(delta+8.0))   
        return 3.0 * A1 * A2 * pow(1.0+z, (4.0-delta)/(delta+8.0)) * pow(xief, 2.0) * pow(xiBf, 0.5) * pow(eta, delta/(2.0*(delta+8.0))) * pow(E1, 4.0/(delta+8.0)) * pow(t, -12.0/(delta+8.0)) * 1.0 / constants.GHz * 1.0 / 1.48

    def Ec_bb_coc(xiBf, xief, E1, alpha, eta, D, z, t, delta, dtheta):
        # This module calculates the cut off frecuency and give the value in Hz
        A1 = pow(2.0, 0.5) * constants.e * pow(constants.me, 1.0) / (128.0 * pow(np.pi, 0.5) * pow(constants.mp, 3.0/2.0) * pow(constants.sigmaT, 2.0))
        A2 = pow(3.0/(32.0*np.pi*constants.mp), -4.0/(delta+8.0))
        xf = xff_coc(xiBf, xief, E1, alpha, eta, D, z, t, delta, dtheta)
        t0 = 0.0
        return A1 * A2 * pow(1.0+z, (delta-4.0)/(delta+8.0)) * pow(1.0+xf, -2.0) * pow(xiBf, -3.0/2.0) * pow(eta, -(16.0+3.0*delta)/(2.0*(delta+8.0))) * pow(E1, -4.0/(delta+8.0)) * pow(t-t0, -2.0*(delta+2.0)/(delta+8.0)) * 1.0 / constants.KeV * 1.48

    def Fmax_bb_coc(xiBf, xief, E1, alpha, eta, D, z, t, delta, dtheta):
        # This module calculates the cut off frecuency and give the value in Hz
        A1 = pow(2.0, 0.5) * constants.me * constants.sigmaT / (12.0 * pow(np.pi, 0.5) * pow(constants.mp, 0.5) * constants.e)
        A2 = pow(3.0/(32.0*np.pi*constants.mp), -delta/(delta+8.0))
        return A1 * A2 * pow(1.0+z, -4.0*(delta+2.0)/(delta+8.0)) * pow(xiBf, 0.5) * pow(eta, (3.0*delta+8.0)/(2.0*(delta+8.0))) * pow(D, -2.0) * pow(E1, 8.0/(delta+8.0)) * pow(t, 3.0*delta/(delta+8.0)) * 1.0 / constants.mJy * 1.0 / pow(1.48, 2.0)

    def Em_ab_coc(xiBf, xief, gama0b, E1, alpha, eta, D, z, t, delta1):
        # This module calculates the characteristic frecuency and give the value in Hz
        A1 = pow(2.0, 0.5) * constants.e * pow(constants.mp, 2.5) / (pow(np.pi, 0.5) * pow(constants.me, 3.0)) * pow((alpha-2.0)/(alpha-1.0), 2.0)
        A2 = pow(3.0/(32.0*np.pi*constants.mp), 4.0/(delta1+6.0))
        return 3.0 * A1 * A2 * pow(1.0+z, (6.0-delta1)/(delta1+6.0)) * pow(xief, 2.0) * pow(xiBf, 0.5) * pow(eta, (delta1-2.0)/(2.0*(delta1+6.0))) * pow(E1, 4.0/(delta1+6.0)) * pow(t, -12.0/(delta1+6.0)) * 1.0 / constants.GHz

    def Ec_ab_coc(xiBf, xief, gama0b, E1, alpha, eta, D, z, t, delta1):
        # This module calculates the characteristic frecuency and give the value in Hz
        A1 = pow(2.0, 0.5) * constants.e * pow(constants.me, 1.0) / (128.0 * pow(np.pi, 0.5) * pow(constants.mp, 3.0/2.0) * pow(sigmaT, 2.0))
        A2 = pow(3.0/(32.0*np.pi*constants.mp), -4.0/(delta1+6.0))
        return A1 * A2 * pow(1.0+z, (delta1-6.0)/(delta1+6.0)) * pow(xiBf, -3.0/2.0) * pow(eta, -(3.0*delta1+10.0)/(2.0*(delta1+6.0))) * pow(E1, -4.0/(delta1+6.0)) * pow(1.0+xf, -2.0) * pow(t, -(2.0*delta1)/(delta1+6.0)) * 1.0 / constants.KeV

    def Fmax_ab_coc(xiBf, xief, gama0b, E1, alpha, eta, D, z, t, delta1):
        # This module calculates the cut off frecuency and give the value in Hz
        A1 = pow(2.0, 0.5) * constants.me * constants.sigmaT / (12.0 * pow(np.pi, 0.5) * pow(constants.mp, 0.5) * constants.e)
        A2 = pow(3.0/(32.0*np.pi*constants.mp), (2.0-delta1)/(delta1+6.0))
        return A1 * A2 * pow(1.0+z, (12.0- 2.0*delta1)/(delta1+6.0)) * pow(xiBf, 0.5) * pow(eta, (3.0*delta1+2.0)/(2.0*(delta1+6.0))) * pow(D, -2.0) * pow(E1, 8.0/(delta1+6.0)) * pow(t, -3.0*(2.0-delta1)/(delta1+6.0)) * 1.0 / constants.mJy
    
    # Fit Model
    td_jet = pm.Deterministic('tdj', tdx_jet(xiBf, xief, E_off, alpha, eta, D, z, tr6, delta, dtheta))
    td_coc = pm.Deterministic('tdc', tdx_coc(xiBf, xief, E0, alpha, eta, D, z, tr6, delta, dtheta))
    
    if (tr6 < td_jet).any():
        print("hello")
        Egammam_sc = 6 * constants.GHz
        #Egammam_sc = 6 * constants.KeV
        E_m_jet = pm.Deterministic('E_m_jet', Em_bb_jet(xiBf, xief, gama0b, E_off, alpha, eta, D, z, tr6, delta, dtheta, theta_j) * constants.GHz)
        E_c_jet = pm.Deterministic('E_c_jet', Ec_bb_jet(xiBf, xief, E_off, alpha, eta, D, z, tr6, delta, dtheta, theta_j) * constants.KeV)
        Fmax_jet = pm.Deterministic('F_max_jet', Fmax_bb_jet(xiBf, xief, E_off, alpha, eta, D, z, tr6, delta, dtheta, theta_j) * constants.mJy * 1.0)
        Fm_sc_Radio6 = pm.Deterministic('Fm_sc_Radio6', Fmax_jet*pow(Egammam_sc/E_m_jet, -(alpha-1.0)/2.0)*1.0/constants.micJy)
        Fm_sc_Radio6_obs = pm.Normal('Fm_sc_Radio6_obs', mu=Fm_sc_Radio6, sd=sigma, observed=F_r6g)
    else:
        print("world")
        Egammam_sc_R= 6 * constants.GHz
        #Egammam_sc = 6 * constants.KeV
        E_m_R_jet = pm.Deterministic('E_m_R_jet', Em_ab_jet(xiBf, xief, gama0b, E_off, alpha, eta, D, z, tr6, delta)*constants.GHz)
        E_c_R_jet = pm.Deterministic('E_c_R_jet', Ec_ab_jet(xiBf, xief, E_off, alpha, eta, D, z, tr6, delta, dtheta)*constants.KeV)
        Fmax_R_jet = pm.Deterministic('F_max_R_jet', Fmax_ab_jet(xiBf, xief, E_off, alpha, eta, D, z, tr6, delta, dtheta)*constants.mJy*1.0)
        Fm_sc_Radio6 = pm.Deterministic('Fm_sc_Radio6', Fmax_R_jet*pow(Egammam_sc_R/E_m_R_jet, -(alpha-1.0)/2.0)*1.0/constants.micJy)
        Fm_sc_Radio6_obs = pm.Normal('Fm_sc_Radio6_obs', mu=Fm_sc_Radio6, sd=sigma, observed=F_r6g)

    if (tr6 < td_coc).any():
        print("foo")
        Egammam_sc = 6 * constants.GHz
        #Egammam_sc = 6 * constants.KeV
        E_m_coc = pm.Deterministic('E_m_coc', Em_bb_coc(xiBf, xief, gama0b, E0, alpha, eta, D, z, tr6, delta, dtheta)*constants.GHz)
        E_c_coc = pm.Deterministic('E_c_coc', Ec_bb_coc(xiBf, xief, E0, alpha, eta, D, z, tr6, delta, dtheta)*constants.KeV)
        Fmax_coc = pm.Deterministic('Fmax_coc', Fmax_bb_coc(xiBf, xief, E0, alpha, eta, D, z, tr6, delta, dtheta)*constants.mJy)
        Fm_sc_Radio6_c = pm.Deterministic('Fm_sc_Radio6_c', Fmax_coc*pow(Egammam_sc/E_m_coc, -(alpha-1.0)/2.0)*1.0/constants.micJy)
        Fm_sc_Radio6_obs = pm.Normal('Fm_sc_Radio6_obs_c', mu=Fm_sc_Radio6_c, sd=sigma, observed=F_r6g)
    else:
        print("bar")
        Egammam_sc_R= 6 * constants.GHz
        #Egammam_sc = 6 * constants.KeV
        E_m_R_coc = pm.Deterministic('E_m_R_coc', Em_ab_coc(xiBf, xief, gama0b, E0, alpha, eta, D, z, tr6, delta)*constants.GHz)
        E_c_R_coc = pm.Deterministic('E_c_R_coc', Ec_ab_coc(xiBf, xief, gama0b, E0, alpha, eta, D, z, tr6, delta)*constants.KeV)
        Fmax_R_coc = pm.Deterministic('Fmax_R_coc', Fmax_ab_coc(xiBf,xief,gama0b,E0,alpha,eta,D,z,tr6,delta)*a.mJy)
        Fm_sc_Radio6_c = pm.Deterministic('Fm_sc_Radio6_c', Fmax_R_coc*pow(Egammam_sc_R/E_m_R_coc, -(alpha-1.0)/2.0)*1.0/constants.micJy)
        Fm_sc_Radio6_obs = pm.Normal('Fm_sc_Radio6_obs_c', mu=Fm_sc_Radio6_c, sd=sigma, observed=F_r6g)

    # Step Definition
    step1 = pm.NUTS(target_accept=0.99)

    # Sampling
    #trace = pm.sample(niter, step=[step1], init="adapt_diag",tune=tune, random_seed = 123)
    trace = pm.sample(niter)
    aa    = pm.backends.tracetab.trace_to_dataframe(trace, varnames={r'$\tilde{E} \ (10^{49}erg)$',r'$E_{off} \ (10^{49}erg)$',r'$\Delta\theta \ (deg)$',r'$n \ (10^{-4}cm^{-3})$',r'$\theta_j \ (deg)$',r'$p$',r'$\alpha_S$',r'$\epsilon_B\ (10^{-4})$',r'$\epsilon_e \ (10^{-1}) $'})
    aa.to_csv(path_or_buf = "Output/Radio6GHz/output.csv", sep="\t")
    summary = pm.stats.summary(trace,varnames={r'$\tilde{E} \ (10^{49}erg)$',r'$E_{off} \ (10^{49}erg)$',r'$\Delta\theta \ (deg)$',r'$n \ (10^{-4}cm^{-3})$',r'$\theta_j \ (deg)$',r'$p$',r'$\alpha_S$',r'$\epsilon_B\ (10^{-4})$',r'$\epsilon_e \ (10^{-1}) $'})
    summary.to_csv(path_or_buf = "Output/Radio6GHz/summary.csv", sep="\t")
    print(summary)

hello
foo


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [$\sigma$, $\epsilon_e \ (10^{-1}) $, $\epsilon_B\ (10^{-4})$, $\alpha_S$, $p$, $\theta_j \ (deg)$, $\Delta\theta \ (deg)$, $\tilde{E} \ (10^{49}erg)$, $E_{off} \ (10^{49}erg)$, $n \ (10^{-4}cm^{-3})$]


JoblibValueError: JoblibValueError
___________________________________________________________________________
...........................................................................
/usr/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

...........................................................................
/usr/lib/python2.7/ru

In [ ]:
pm.traceplot(trace)

In [ ]:
Radio6GHz_model.free_RVs